In [4]:
import tensorflow as tf
import numpy as np
import cv2
from glob import glob

In [2]:
# best fiber extraction: Threshold Binary70 + Guassian Otsu's Thresholding
# best centre line: Threshold Binary70inv + closing + skel + remove small 1 contours
# 先threshould 根据边界框框crop crop完了直接用binary的图片提取12个trajectory 然后每个trajetory和GT的进行
# overlay 计算不同粗细的precision
# 还有一个metric就是RMSE计算pred每个pixel的位置和最近的GT的pixel的欧拉距离 但是不是很好算 要么就算了。
# 应该先提取fiber 然后croped了再分别不同trajectory再识别中心线 没有合适的metric就是说。

In [2]:
# 提取fiber部分
def extract_cline_fiber(pic):
    ret,thresh1 = cv2.threshold(pic, 210, 255, cv2.THRESH_BINARY_INV)
    blur = cv2.GaussianBlur(thresh1, (5,5), 0)#blur的size可以修改
    _, th3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return th3

In [6]:
# 提取12个图片的黑白版本
files = glob('12 fiber pics\\*.*')
for file in files:
    name = list(file.split('\\'))[-1]
    pic = cv2.imread(file, 0)
    new_pic = extract_cline_fiber(pic)
    cv2.imwrite('12 fiber pics\\extracted_' + name, new_pic)

In [8]:
# 这里要搞一个根据框截图图片的function 从image变成croped

In [3]:
# 输入crop的binary图片，输出12个trajectory叠在一起的图和对应的
def get_traj(pic):
    contours, hierarchy = cv2.findContours(pic, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
    lefts = []
    trajs = np.zeros([pic.shape[0],pic.shape[1],12])
    
    for i in range(len(sorted_contours)):
        c = sorted_contours[i]
        area = cv2.contourArea(c)
        if (area < 100 and i < 12) or (area >= 100 and i >= 12):
            print('Disconnection may exist! Please check!')
        elif area >= 100 and i < 12:
            traj = cv2.drawContours(np.zeros(pic.shape),[c],0,255,-1)
            trajs[:,:,i] = traj
            l = np.where(traj!=0)[1].min()
            lefts.append(l)
        else:
            pass
    
    ordered = np.zeros(trajs.shape)
    n = 0
    for i in np.argsort(np.array(lefts)):
        ordered[:,:,n] = trajs[:,:,i]
        n+=1
        
    return ordered

In [4]:
# 根据evaluate的曲线位置（0-11）和evaluate的range（1,2,3意味着几倍）生成取样区域
def eva_area(img, pos=1, rang=1):
    
    num = pos//3
    loc = pos%3
    center = 130+300*num
    pic = np.zeros([630, 1160])
    if loc == 2:
        R = 25
    elif loc == 1:
        R = 50
    elif loc == 0:
        R = 100
    
    thick = np.round(rang*6.5).astype(int)
    cv2.circle(pic, (center,130), R, 255, thick)
    cv2.rectangle(pic, (center-R-thick,130), (center+R+thick,130+R+thick), 0, -1)
    cv2.line(pic, (center-R,130), (center-R,130+470), 255, thick) 
    cv2.line(pic, (center+R,130), (center+R,130+470), 255, thick) 
    
    if img.shape[1]/img.shape[0] != 116/63:
        print('Image shape is not defult:', img.shape)
    pic = np.round(cv2.resize(pic, (img.shape[1],img.shape[0]))/255.)*255
    
    return pic

In [5]:
# 输入一个trajectory图片和一个GT图片 输出precision
# 根据一个trajectory和一个GT的不同range计算precision
def calc_pres(GT, pred):
    GT = np.round(GT/255.).astype(int)
    pred = np.round(pred/255.).astype(int)
    tot = np.count_nonzero(pred)
    TP = np.count_nonzero(GT&pred)

    return TP/tot

In [39]:
# 计算整个trajectory的AP
# 输入12个叠在一起的trajectory图片和某trajectory的位置 输出从range1-5曲线下的面积
def get_AP(ordered, pos):
    AP = 0
    pred = ordered[:,:,pos]
    for i in range(5):
        GT = eva_area(pred, pos, i+1)
        Pres = calc_pres(GT, pred)
        AP += Pres*0.2
        if i == 4 and Pres < 0.99:
            print(str(pos), 'This trajectory requires larger range.')
    return AP

In [40]:
# 计算某个trajectory单个section的AP
# 输入12个叠在一起的trajectory图片，某trajectory的位置和哪部分线段（前，中，后） 输出从range1-5曲线下的面积
def get_AP_section(ordered, pos, section='arc'):
    pred = ordered[:,:,pos]
    AP = 0
    height, width = pred.shape
    center = 130+300*(pos//3)
    for i in range(5):
        GT = eva_area(pred, pos, i+1)
        if section == 'arc':
            pred_s = pred[:int(13/63*height), :]
            GT_s = GT[:int(13/63*height), :]
        elif section == 'before':
            pred_s = pred[int(13/63*height):, :center]
            GT_s = GT[int(13/63*height):, :center]
        elif section == 'after':
            pred_s = pred[int(13/63*height):, center:]
            GT_s = GT[int(13/63*height):, center:]
        else:
            print('Please input before, arc or after in section.')
#         cv2.imwrite('GTtest.png', GT_s)
#         cv2.imwrite('Predtest.png', pred_s)
        Pres = calc_pres(GT_s, pred_s)
        AP += Pres*0.2
        if i == 4 and Pres < 0.99:
            print(str(pos), 'This trajectory requires larger range.')
    return AP

In [41]:
# 可视化单个trajectory AP
def view_AP(ordered, pos, rang=1):
    blk = np.zeros([ordered.shape[0],ordered.shape[1]])
    pred = ordered[:,:,pos]
    GT = eva_area(pred, pos, rang)
    pred = pred.reshape([ordered.shape[0],ordered.shape[1],1])
    GT = GT.reshape([ordered.shape[0],ordered.shape[1],1])
    blk = blk.reshape([ordered.shape[0],ordered.shape[1],1])
    
    overlay = np.concatenate([blk, pred, GT], axis=-1)
    return overlay   

In [44]:
pos = 1 #change trajectory position here (0-11)
sec = 'before' # change trajectory section here (before, arc, after)
pic = cv2.imread('croped.bmp', 0)
ordered = get_traj(pic)
print('trajectory %s AP:' %pos, get_AP(ordered, 1))
print('trajectory %s section %s AP:' %(pos, sec), get_AP_section(ordered, pos, section=sec))
overlay = view_AP(ordered, pos, rang=1)
cv2.imwrite('overlay.png', overlay)

trajectory 1 AP: 0.9007514831905077
trajectory 1 section before AP: 0.8811562385656788


True

In [7]:
# 从左到右四个是不同的参数 所以这部分用不到了
# 输入整个trajectory图片和该trajectory的位置 输出从range1-5曲线下的面积
# def get_meanAP(ordered, trajectory='outer'):
#     if trajectory == 'outer':
#         poses = np.array([0,3,6,9])
#     elif trajectory == 'middle':
#         poses = np.array([0,3,6,9])+1
#     elif trajectory == 'inner':
#         poses = np.array([0,3,6,9])+2
#     else:
#         print('Please define trajectory inner, middle or outer.')
#     APs = []
#     for pos in poses:
#         APs.append(get_AP(ordered[:,:,pos], pos))
#     return sum(APs)/len(APs)

In [6]:
# 旧的版本 现在应该没用了
# 根据evaluate的曲线位置和evaluate的range生成取样区域
# def eva_area(img, num=1, loc=1, thick=1):
#     center = 130+300*(num-1)
#     pic = np.zeros([630, 1160])
#     if loc == 1:
#         R = 25
#     elif loc == 2:
#         R = 50
#     elif loc == 3:
#         R = 100
#     else:
#         print('Please choose location 1, 2 or 3.')
        
#     cv2.circle(pic, (center,130), R, 255, thick)
#     cv2.rectangle(pic, (center-R-thick,130), (center+R+thick,130+R+thick), 0, -1)
#     cv2.line(pic, (center-R,130), (center-R,130+470), 255, thick) 
#     cv2.line(pic, (center+R,130), (center+R,130+470), 255, thick) 
    
#     if img.shape[1]/img.shape[0] != 116/63:
#         print('Image shape is not defult:', img.shape)
#     pic = np.round(cv2.resize(pic, (img.shape[1],img.shape[0]))/255.)*255
    
#     return pic

In [2]:
# 用来生成中心线的function
# 源代码来源 http://opencvpython.blogspot.com/2012/05/skeletonization-using-opencv-python.html
# def Skeletonization(img):
#     size = np.size(img)
#     skel = np.zeros(img.shape,np.uint8)
#     element = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
#     done = False

#     while( not done):
#         eroded = cv2.erode(img,element)
#         temp = cv2.dilate(eroded,element)
#         temp = cv2.subtract(img,temp)
#         skel = cv2.bitwise_or(skel,temp)
#         img = eroded.copy()

#         zeros = size - cv2.countNonZero(img)
#         if zeros==size:
#             done = True

#     return skel

In [48]:
# 好像没用了 只取最大面积的12个部分的话
# 移除只有一个pixel的噪音
# def remove_1_noise(pic):
#     contours, hierarchy = cv2.findContours(pic, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     threshold_blobs_area = 0

#     for i in range(1, len(contours)):
#         index_level = int(hierarchy[0][i][1])
#         if index_level <= i:
#             cnt = contours[i]
#             area = cv2.contourArea(cnt)
#             if area <= threshold_blobs_area:
#                 cv2.drawContours(pic, [cnt], -1, 0, -1, 1)
#     return pic

In [290]:
pic = cv2.imread('o1.bmp',0)
print(pic.shape)
skl = Skeletonization(pic)
# skl = remove_1_noise(skl)
cv2.imwrite('o1_clin.bmp', skl)

(630, 1160)


True

In [292]:
# for i in range(ordered.shape[2]):
#     traj = ordered[:,:,i]
#     print(traj.shape)
#     cline = Skeletonization(traj)
#     cv2.imwrite('traj_'+str(i) + '.png', traj)
#     cv2.imwrite('cline_'+str(i) + '.png', cline)

In [4]:
# 输入：原始图片 输出：fiber threshold和中心线
def extract_cline_fiber(pic):
    # 提取fiber部分
    ret,thresh1 = cv2.threshold(pic, 210, 255, cv2.THRESH_BINARY_INV)
    blur = cv2.GaussianBlur(thresh1, (5,5), 0)#blur的size可以修改
    _, th3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
#     cv2.imwrite("Guassian Otsu's Thresholding.bmp", th3) 
    
    # 提取centre line部分
    kernel = np.ones((5,5), np.uint8)
    ret,thresh1 = cv2.threshold(pic, 210, 255, cv2.THRESH_BINARY_INV)
    closing = cv2.morphologyEx(thresh1, cv2.MORPH_CLOSE, kernel)
    skel = Skeletonization(closing)
    skel = remove_1_noise(skel)
    
    return th3, skel

In [5]:
# 一键提取center line和fiber的图
pic = cv2.imread('220704.bmp',0)
th3, skel = extract_cline_fiber(pic)
cv2.imwrite('220704fibre.bmp', th3)
cv2.imwrite('220704cline.bmp', skel)

True

In [7]:
def get_contour_areas(contours):
    # returns the areas of all contours as list
    all_areas = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        all_areas.append(area)
    return all_areas

In [15]:
# # 对提取的截取好的（一定和GT重合的）fiber图片进行contour分每个trajectory
# def get_trajectory(croped):
#     edged = cv2.Canny(np.uint8(croped), 50, 200)
#     contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
#     sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
#     areas = get_contour_areas(sorted_contours)
#     blk_image = np.zeros(croped.shape)
#     if len(areas) > 12:
#         idx = 12-len(areas)
#         if np.any(np.array(areas[idx:]) > 100):
#             print('Disconnetion may exist, please check!')
#         else:
#             n = 0
#             for c in sorted_contours:
# #                 print(c)
#                 cv2.drawContours(blk_image, c, -1, 170, cv2.FILLED)
# #                 cv2.drawContours(blk_image,[c],0,(0,255,0),-1)
# #                 cv2.drawContours(blk_image, c, 0, 170, cv2.FILLED)
# #                 cv2.waitKey(0)
#                 cv2.imwrite('contour_' + str(n) + '.png', blk_image)
#                 n+=1
#     elif len(areas) < 12:
#         print('Only %d trajectory detected, please check!' %len(areas))
#     print (get_contour_areas(sorted_contours))
#     print ("Number of contours found = ", len(contours))

In [38]:
# 尝试画出来area的新的对提取的截取好的（一定和GT重合的）fiber图片进行contour分每个trajectory
def get_trajectory(croped):
#     print(np.unique(croped))
    edged = cv2.Canny(np.uint8(croped), 50, 200)
    ret,thresh1 = cv2.threshold(np.uint8(croped), 210, 255, cv2.THRESH_BINARY_INV)
    cv2.imwrite('thistimethresh1.png', thresh1)
#     contours, hierarchy = cv2.findContours(thresh1.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
    areas = get_contour_areas(sorted_contours)
    blk_image = np.zeros(croped.shape)
    if len(areas) > 12:
        idx = 12-len(areas)
        if np.any(np.array(areas[idx:]) > 100):
            print('Disconnetion may exist, please check!')
        else:
            n = 0
            for c in sorted_contours:
#                 print(c)
                cv2.drawContours(blk_image, c, -1, 170, cv2.FILLED)
#                 cv2.drawContours(blk_image,[c],0,(0,255,0),-1)
#                 cv2.drawContours(blk_image, c, 0, 170, cv2.FILLED)
#                 cv2.waitKey(0)
                cv2.imwrite('contour_' + str(n) + '.png', blk_image)
                n+=1
    elif len(areas) < 12:
        print('Only %d trajectory detected, please check!' %len(areas))
    print (get_contour_areas(sorted_contours))
    print ("Number of contours found = ", len(contours))

In [39]:
get_trajectory(croped)

[8682.5, 8337.5, 8050.0, 7719.0, 7100.0, 6788.5, 6785.0, 6679.5, 6666.5, 6543.5, 6455.5, 5211.5, 13.0, 10.0, 4.0]
Number of contours found =  15


In [ ]:
# 根据生成的标准图片eva_area和输入图片计算该选中的area下的

True

In [40]:
# 这里是随便截取一下打印提取的图片 看能不能用来跟上面的eva area重合一下
# height, w = th3.shape
l,u = 100, 450
h = 1000
w = int(h/63*116)
croped = th3[u:u+h, l:l+w]
croped = np.round((cv2.resize(croped, (1160, 630)))/255.).astype(int)*255
# print(croped)
o1 = eva_area(croped,num=1, loc=3, thick=10).astype(int)
np.unique(o1)
cv2.imwrite('o1.bmp', o1)
cv2.imwrite('croped.bmp', croped)
added_image = cv2.addWeighted(croped,0.4,o1,0.3,0)
cv2.imwrite('overlay.bmp', added_image)

True

In [41]:
np.unique(croped)

array([  0, 255])

In [46]:
image = cv2.imread('d3d10597-e629-4f08-853b-75472ae0de06.bmp',0)
kernel = np.ones((5,5), np.uint8)
ret,thresh1 = cv2.threshold(image, 70, 255, cv2.THRESH_BINARY_INV)
# Closing - for removing noise
# blur = cv2.GaussianBlur(image, (5,5), 0)
# _, th3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
closing = cv2.morphologyEx(thresh1, cv2.MORPH_CLOSE, kernel)
cv2.imwrite('Closing.bmp', closing)

skel = Skeletonization(closing)
cv2.imwrite('Closingskel.bmp', skel)

contours, hierarchy = cv2.findContours(skel, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
threshold_blobs_area = 0

for i in range(1, len(contours)):
    index_level = int(hierarchy[0][i][1])
    if index_level <= i:
        cnt = contours[i]
        area = cv2.contourArea(cnt)
        print(area)
        if area <= threshold_blobs_area:
            cv2.drawContours(skel, [cnt], -1, 0, -1, 1)
cv2.imwrite('removednoiseClosingskel.bmp', skel)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
15.5
1.5
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
3.0
8.5
14.5
0.0
8.5
0.0
0.0
0.0
0.0
0.0
0.0
5.0
5.0
0.0
10.5
0.0
0.0
0.0
0.0
0.0
0.0
0.5
0.0
0.0
0.0
33.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
46.0
0.0
11.5
11.0
0.0
24.0
35.5
0.0
16.0
40.0
0.0
0.0
0.0
0.0
3.5
3.5
2.0
0.0
0.0
32.0
0.0
0.0
0.0
0.0
23.0
14.5
0.0
80.0
0.0
0.0
6.0
1.0
0.0
0.0
0.0
45.0
110.0
0.0
0.0
48.5
33.0
25.0
22.5
107.0
35.0
0.0
0.0
0.0
0.0
0.0
7.0
2.0
0.0
0.0
30.0
74.0
0.0
40.0
8.5
0.5
0.0
0.0
61.0
0.0
95.5
39.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
41.0
0.0
74.5
0.0
16.5
0.0
0.0
0.0
58.0
0.0
2.5
0.0
0.0
0.0
2.5
0.0
69.0
0.0
0.0
0.0
0.0
1.0
0.0
71.0
55.5
0.0
3.0
0.0
0.0
0.0
48.0
0.0
0.0
84.0
97.0
0.0
33.0
0.0
2.5
82.0
107.0
0.0
0.0
14.5
10.0
0.0
68.0
47.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
157.0
0.0
0.0
0.0
0.0
0.0
14.0
33.0
32.5
0.0
2.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
12.5
0.0
0.0
0.0
0.0
0.0
24.0
0.0
6.0
0.0
205.5
81.0
111.5
0.0
74.0
83.5
0.0
290.0
0.0
206.0
1

True

In [12]:
img = cv2.imread('d3d10597-e629-4f08-853b-75472ae0de06.bmp',0)
img = cv2.medianBlur(img,5)
th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)
cv2.imwrite("ADAPTIVE_THRESH_GAUSSIAN Thresholding.bmp", th3) 

True

In [32]:
image = cv2.imread('Threshold Binary70.bmp',0)
# It's good practice to blur images as it removes noise
image = cv2.GaussianBlur(image, (3, 3), 0)

# Using adaptiveThreshold
thresh = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
                               cv2.THRESH_BINARY, 3, 5) 
cv2.imwrite("Adaptive Mean Thresholding.bmp", thresh) 
# cv2.waitKey(0) 

_, th2 = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
cv2.imwrite("Otsu's Thresholding.bmp", th2) 
# cv2.waitKey(0) 

# Otsu's thresholding after Gaussian filtering
blur = cv2.GaussianBlur(image, (5,5), 0)
_, th3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
cv2.imwrite("Guassian Otsu's Thresholding.bmp", th3) 

# th3 = cv.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY,11,2)
# cv2.waitKey(0) 

True

In [12]:
img = cv2.imread('Threshold Binary70.bmp',0)

size = np.size(img)
skel = np.zeros(img.shape,np.uint8)

ret,img = cv2.threshold(img,127,255,cv2.THRESH_BINARY_INV)
# cv2.imwrite('test.bmp', img)
# img = cv2.Canny(img, 300, 120)
element = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
done = False

while( not done):
    eroded = cv2.erode(img,element)
    temp = cv2.dilate(eroded,element)
    temp = cv2.subtract(img,temp)
    skel = cv2.bitwise_or(skel,temp)
    img = eroded.copy()

    zeros = size - cv2.countNonZero(img)
    if zeros==size:
        done = True

cv2.imwrite('skel.bmp', skel)
# cv2.imshow("skel",skel)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

True

In [8]:
import cv2
import numpy as np

img = cv2.imread('out1.bmp',0)
size = np.size(img)
skel = np.zeros(img.shape,np.uint8)

ret,img = cv2.threshold(img,127,255,0)
element = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
done = False

while( not done):
    eroded = cv2.erode(img,element)
    temp = cv2.dilate(eroded,element)
    temp = cv2.subtract(img,temp)
    skel = cv2.bitwise_or(skel,temp)
    img = eroded.copy()

    zeros = size - cv2.countNonZero(img)
    if zeros==size:
        done = True

cv2.imshow("skel",skel)
cv2.waitKey(0)
cv2.destroyAllWindows()